In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load dataset
df = pd.read_csv('house-prices.csv')

# Data preprocessing
# Assuming 'Price' is the target column and we convert it to categories
bins = [0, 100000, 300000, 500000, np.inf]
labels = ['Low', 'Medium', 'High', 'Very High']
df['Price Category'] = pd.cut(df['Price'], bins=bins, labels=labels)

# Encoding categorical target variable
label_encoder = LabelEncoder()
df['Price Category'] = label_encoder.fit_transform(df['Price Category'])

# Splitting features and target
X = df.drop(columns=['Price', 'Price Category'])  # Drop target column and original price
y = df['Price Category']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert categorical features to numerical using one-hot encoding
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Align columns in training and testing sets
# This ensures both sets have the same columns after one-hot encoding
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)

# Impute missing values after one-hot encoding
X_train.fillna(X_train.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

# Standardize features
scaler = StandardScaler() # Initialize scaler here
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define MLP model
def create_mlp(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(4, activation='softmax')  # 4 output classes
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model
    # Create and train the model
mlp_model = create_mlp(X_train.shape[1])
history = mlp_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16)

# Fine-tuning with additional training
def fine_tune_model(model, X_train, y_train, X_test, y_test, epochs=20):
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=16)
    return model

# Fine-tune the model
mlp_model = fine_tune_model(mlp_model, X_train, y_train, X_test, y_test)

# Evaluate the model
test_loss, test_acc = mlp_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - accuracy: 0.1876 - loss: 1.4186 - val_accuracy: 0.8846 - val_loss: 1.1315
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8905 - loss: 1.0097 - val_accuracy: 0.8846 - val_loss: 0.8710
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9322 - loss: 0.7794 - val_accuracy: 0.8846 - val_loss: 0.6289
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9277 - loss: 0.5447 - val_accuracy: 0.8846 - val_loss: 0.4628
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9318 - loss: 0.3566 - val_accuracy: 0.8846 - val_loss: 0.3894
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9212 - loss: 0.3257 - val_accuracy: 0.8846 - val_loss: 0.3647
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9089 - loss: 0.3299 - val_accuracy: 0.8846 - val_loss: 0.3483
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8916 - loss: 0.3561 - val_accuracy: 0.8846 - val_loss: 0.3273
